# Taller de Física Computacional

# Sesión 2: Empezando a programar

## Funciones definidas por intervalo

Ahora vamos a integrar un poco todo eso que vimos en la Sesión 1. Vamos a escribir algunas funciones matemáticas. Epecemos por la función valor absoluto:
$$
|x| =
\left\{
	\begin{array}{ll}
		x  & \mbox{if } x \geq 0 \\
		-x & \mbox{if } x < 0
	\end{array}
\right.
$$

In [1]:
def abs_val(x):
    if x >= 0.0:
        return x
    else:
        return -x

In [2]:
abs_val(-5.0)

5.0

In [3]:
abs_val(5.0)

5.0

In [4]:
abs_val(0.0)

0.0

## La factorial

Construyamos una función que nos devuelva la factorial de un número
$$ n! = 1 \times 2 \times 3 \times\cdots\times n$$

In [5]:
def factorial(n):
    fact = 1
    for i in range(2,n+1):
        fact = fact * i
    return fact

In [6]:
factorial(2)

2

In [7]:
factorial(45)

119622220865480194561963161495657715064383733760000000000

In [8]:
factorial(0) # funciona!!! porqué?

1

In [9]:
factorial(3.5)

TypeError: 'float' object cannot be interpreted as an integer

Acá va una versión recursiva:

In [10]:
def factorial_rec(n):
    if n >= 1 :
        return n*factorial_rec(n-1)
    else:
        return 1

In [11]:
factorial_rec(5)

120

In [12]:
factorial(45)

119622220865480194561963161495657715064383733760000000000

In [13]:
factorial_rec(3.5) # OJO

13.125

In [14]:
factorial(-5) # OJO

1

Comparemos con la implementación de factorial de python en el paquete `math`:

In [15]:
import math

In [16]:
math.factorial(5)

120

In [17]:
math.factorial(45)

119622220865480194561963161495657715064383733760000000000

In [18]:
math.factorial(3.5)

ValueError: factorial() only accepts integral values

In [19]:
math.factorial(-5)

ValueError: factorial() not defined for negative values

- ¿Qué diferencias puedes encontrar entre nuestras implementaciones y las del paquete `math`?
- ¿En qué podría mejorarse nuestra implementación?
- ¿Podrías proponer una implementación mejorada?

## Evaluando una serie 

Ahora usemos nuestra factorial como bloque para construir una función que evalúe de la expansión de Taylor a orden $n$ de la función exponencial:
$$
e^x = 1 + \frac{x}{1!} + \frac{x^2}{2!} + \frac{x^3}{3!} + \cdots + \frac{x^n}{n!} = \sum_{i=0}^n \frac{x^i}{i!}
$$

In [20]:
def exp_approx(x,n):
    resultado = 0
    for i in range(n+1):
        resultado += x**i / factorial(i)
    return resultado

Para comparar podemos usar la función exponencial que incluye python

In [21]:
exp(3.0)

NameError: name 'exp' is not defined

pero como vemos no está definida en en lenguaje, debemos cargar un **paquete** que contiene todas las funciones trascendentales

In [22]:
math.exp(3.0)

20.085536923187668

In [23]:
exp_approx(3.0,6)

19.412499999999998

In [24]:
exp_approx(3.0,35)

20.08553692318766

podemos ser un poco más ambiciosos y pedir que la función nos devuelva una **función** que sea la aproximación de orden $n$

In [25]:
def exp_approx(n):
    def approx(x):
        resultado = 0
        for i in range(n+1):
            resultado += x**i / factorial(i)
        return resultado
    return approx

In [26]:
app20 = exp_approx(20)

In [27]:
app20(3.0)

20.085536922950844

Te sugerimos desarrollar implementaciones que evalúen las series de Taylor hasta roden $n$ para las funciones $\sin$ y $\cos$.

## Evaluando una integral definida

Así como podemos devolver funciones como resultado podemos consumir funciones como argumento. Armemos una función que me devuelva el valor de la integral definida de una función arbitraria $f$ en el intervalo cerrado $[a,b]$. Vamos a utilizar una aproximación numérica de la integral definida utilizando una suma finita como aproximación. Teniendo en cuenta que
$$
\int_a^b f(\tau)d\tau = \lim_{n \rightarrow \infty} \sum_i= f(c_i) \Delta x
$$
con una partición homogénea del intervalo $[a,b]$ tal que
$$
c_i = a + \frac{(b-a)}{n} i
$$
y
$$
\Delta x = \frac{(b-a)}{n}
$$
Una función que calcula esta integral para una función arbitraria es la que sigue:

In [28]:
def integral_definida(f,a,b,n):
    resultado = 0.0
    delta_x = (b-a)/n
    for i in range(n+1):
        c_i = a + i * (b-a)/n
        resultado += f(c_i) * delta_x
    return resultado

probemos calculando una fácil
$$
\int_0^1 exp(x) = e - 1
$$

In [29]:
math.e - 1

1.718281828459045

In [30]:
integral_definida(math.exp,0,1,20000)

1.7183747858627332

Claramente la aproximación a la integral no es muy buena, necesitamos muchísimos intervalos para que funcione, lo que es peor es que el error depende de donde la evaluemos, por ejemplo

In [31]:
integral_definida(math.exp,0,1,20000) - (math.e - 1)

9.295740368808758e-05

In [32]:
integral_definida(math.exp,0,4,20000) - (math.exp(4) - 1)

0.0055599936640149394

Queremos mejorar esto. Podemos hacerlo agregando un control de convergencia, incrementando el número de particiones hasta que una cierta tolerancia sea satizfecha de la siguiente manera:

In [33]:
def integral_definida2(f,a,b,tol):
    
    n = 100 # el valor inicial de particiones
    resultado = 0.0
    delta_x = (b-a)/n
    for i in range(n+1):
        c_i = a + i * (b-a)/n
        resultado += f(c_i) * delta_x
    
    error = tol + 1.0
     
    while error > tol:
        
        n += 100
        resultado_ant = resultado
        
        resultado = 0.0
        delta_x = (b-a)/n
        for i in range(n+1):
            c_i = a + i * (b-a)/n
            resultado += f(c_i) * delta_x
        
        error = abs(resultado_ant-resultado)
        
    print("Hice",n,"particiones con un error de",error)
    return resultado

In [34]:
def integral_definida2(f,a,b,tol):
    
    n = 0 # inicializamos n
    error = 10 * tol # nos aseguramos de que corra al menos una iteración
    resultado = 0.0 # inicializamos el resultado ¿Puece fallar esta inicialización?
     
    while error > tol:
        
        n += 100 # vamos a incrementar n de 100 a 100
        resultado_ant = resultado
        
        resultado = 0.0
        delta_x = (b-a)/n
        for i in range(n+1):
            c_i = a + i * (b-a)/n
            resultado += f(c_i) * delta_x
        
        error = abs(resultado_ant-resultado)
        
    print("Hice",n,"particiones con un error de",error)
    return resultado

In [35]:
integral_definida2(math.exp,0,1,0.0000001) - (math.e - 1)

Hice 43200 particiones con un error de 9.98511109351341e-08


4.303574603370386e-05

In [36]:
integral_definida2(math.exp,0,1,0.00001)  - (math.e - 1)

Hice 4400 particiones con un error de 9.826674196133922e-06


0.00042253942214331985

## Para explorar:

- ¿Porqué si bien la integral converge, no converge al valor correcto?
- ¿Cómo podríamos mejorar la estimación?
- Implementa una nueva rutina que dé una mejor estimación 

## Usando Scipy

Una de las grandes ventajas del lenguaje Python es que existen una enorme variedad de librerías de funciones que resuelven una gran cantidad de problemas. En Python en general es bueno buscar y ver si el problema que estamos tratando de resolver no ha sido ya resuelto por alguien. No *reinventar la rueda*, en la jerga. Y aún en caso de que querramos reimplementar una solución por nuestra cuenta lo que existe puede servirnos para no cometer errores que otros ya han resuelto. 

El paquete [Scipy](https://docs.scipy.org/doc/scipy/reference/index.html) es tan grande que conviene pegar una mirada a la documentación para descubrir sus posibilidades. Claramente la integración numérica está implementada. Utilicemos ahora esa implementación para calcular la integral definida:

In [37]:
from scipy import integrate

In [38]:
integrate.quad(math.exp,0,1)[0]  - (math.e - 1)

2.220446049250313e-16

In [39]:
integrate.quad(math.exp,0,1)

(1.7182818284590453, 1.9076760487502457e-14)

El llamado nos devuelve una tupla con el valor de la integral y una estimación del error. La estimación del error es superior al error real, la idea es que ese sea el caso siempre para que podamos saber cuan correcto es el resultado. Como se puede ver la rutina de Scipy es mucho más rápida y precisa que nuestra implementación *naïve*. 

## Derivada numérica

Construyamos ahora una función que nos devuelva la derivada numérica de una función arbitraria. Podemos utilizar distintos métodos, el más simple es una aproximación de dos puntos que utiliza la definición de la derivada. Teniendo en cuenta que 

$$f'(x) = \lim_{h\rightarrow 0} \frac{f(x+h)-f(x)}{h}$$

Podemos aproximar la derivada de la siguiente forma:

$$f'(x) \approx \frac{f(x+h)-f(x)}{h}$$

Probemos.

In [40]:
def derivada(f,x,h):
    return (f(x+h)-f(x))/h

In [41]:
derivada(math.exp,1,0.001)

2.7196414225332255

In [42]:
math.e

2.718281828459045

In [43]:
derivada(math.exp,1,0.001) - math.e

0.0013595940741804036

In [44]:
derivada(math.exp,1,0.0001) - math.e

0.00013591862387896114

In [45]:
derivada(math.exp,1,0.00001) - math.e

1.3591497672216235e-05

In [46]:
derivada(math.exp,1,0.0000000000000001) - math.e

-2.718281828459045

- ¿Qué pasó ahí? 
- ¿Que estimación del *orden* del error puedes hacer en base a los resultados anteriores?
- ¿Puedes encontrar una explicación de porqué salio tan mal en el último caso?

Una mejor aproximación puede hacerse de la siguiente forma. Teniendo en cuenta que el desarrollo de Taylor de taylor de $f(h+x)$ y $f(x-h)$ son:

$$ f(x+h) = f(x) + hf'(x) + \frac{1}{2}h^2f''(x)+O(h^3)+\cdots$$

$$ f(x-h) = f(x) - hf'(x) + \frac{1}{2}h^2f''(x)-O(h^3)+\cdots$$

Restando las dos ecuaciones tenemos que

$$$f'(x) \approx \frac{f(x+h) - f(x-h)}{2h} $$

In [47]:
def derivada2(f,x,h):
    return (f(x+h)-f(x-h))/2/h

In [48]:
derivada2(math.exp,1,0.001) - math.e

4.53046678838831e-07

In [49]:
derivada2(math.exp,1,0.0001) - math.e

4.53056570037802e-09

- ¿Cual es es orden del error en esta aproximación en relación a $h$?
- ¿Porqué esta estimación es mejor que la anterior?

In [50]:
for i in range(1,19):
    h = 10**(-i)
    print(h,derivada(math.exp,1,h) - math.e,derivada2(math.exp,1,h) - math.e)

0.1 0.14056012641483795 0.00453273548837263
0.01 0.01363682732807936 4.530492366905392e-05
0.001 0.0013595940741804036 4.53046678838831e-07
0.0001 0.00013591862387896114 4.53056570037802e-09
1e-05 1.3591497672216235e-05 5.858691309867936e-11
1e-06 1.3589715694983795e-06 -1.634572477371421e-10
1e-07 1.3994668845995761e-07 5.858735718788921e-11
1e-08 -6.60275079056305e-09 -6.60275079056305e-09
1e-09 2.1544185413446826e-07 -6.60275079056305e-09
1e-10 1.5477094836846561e-06 -6.72736565565657e-07
1e-11 3.263395417318904e-05 1.0429493680685908e-05
1e-12 0.0004323142430382454 0.00021026963811321409
1e-13 -0.00045586417666187984 -0.00045586417666187984
1e-14 -0.009337648373663132 -0.009337648373663132
1e-15 0.3903426404913928 0.16829803556636147
1e-16 -2.718281828459045 -2.718281828459045
1e-17 -2.718281828459045 -2.718281828459045
1e-18 -2.718281828459045 -2.718281828459045


Podemos escribir los resultados de una forma más elegante utilizando el *método* `format` del tipo `string`:

In [51]:
for i in range(1,19):
    h = 10**(-i)
    ed1 = derivada(math.exp,1,h) - math.e
    ed2 = derivada2(math.exp,1,h) - math.e
    print("i = {3:2d}  h = {0:5.2e}  error1 = {1:9.2e}  error2 = {2:9.2e}".format(h,ed1,ed2,i))

i =  1  h = 1.00e-01  error1 =  1.41e-01  error2 =  4.53e-03
i =  2  h = 1.00e-02  error1 =  1.36e-02  error2 =  4.53e-05
i =  3  h = 1.00e-03  error1 =  1.36e-03  error2 =  4.53e-07
i =  4  h = 1.00e-04  error1 =  1.36e-04  error2 =  4.53e-09
i =  5  h = 1.00e-05  error1 =  1.36e-05  error2 =  5.86e-11
i =  6  h = 1.00e-06  error1 =  1.36e-06  error2 = -1.63e-10
i =  7  h = 1.00e-07  error1 =  1.40e-07  error2 =  5.86e-11
i =  8  h = 1.00e-08  error1 = -6.60e-09  error2 = -6.60e-09
i =  9  h = 1.00e-09  error1 =  2.15e-07  error2 = -6.60e-09
i = 10  h = 1.00e-10  error1 =  1.55e-06  error2 = -6.73e-07
i = 11  h = 1.00e-11  error1 =  3.26e-05  error2 =  1.04e-05
i = 12  h = 1.00e-12  error1 =  4.32e-04  error2 =  2.10e-04
i = 13  h = 1.00e-13  error1 = -4.56e-04  error2 = -4.56e-04
i = 14  h = 1.00e-14  error1 = -9.34e-03  error2 = -9.34e-03
i = 15  h = 1.00e-15  error1 =  3.90e-01  error2 =  1.68e-01
i = 16  h = 1.00e-16  error1 = -2.72e+00  error2 = -2.72e+00
i = 17  h = 1.00e-17  er

- ¿Puedes explicar los valores de la tabla?